In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.visualization import ZScaleInterval
from astropy.io import fits
import os
import glob
import matplotlib.cm as cm
import scipy.signal
from scipy.optimize import curve_fit
from astropy.stats import sigma_clip
import time
import sm_phot as sm

from astropy.stats import mad_std
from astropy.stats import sigma_clip
from photutils.utils import calc_total_error
import astropy.stats as stat
from photutils import aperture_photometry, CircularAperture, CircularAnnulus, DAOStarFinder

The following cell contains variables which must be initialized to provide this code with the correct filepaths for each set of images and the correct coordinates for alignment targets and the standard star. This code uses the custom made "sm_phot" module specifically created for this project with a variety of functions for image processing. sm_phot can be downloaded from this repository

In [3]:
#define all initial parameters, file paths, aperture radii, standard star mags etc
#set up for 3 band photometry, for two targets: standard star field and star cluster field
#object and filter names: keep standard star object named 'standard' and use whatever target object name is desired
obj=['standard','NGC7789']
filters=['B','V','R']

#filepaths for science and cal frames
impath='2022oct27/'
biaspath=impath+'cal/bias/'
darkpath=impath+'cal/darks/'
flatpath=impath+'cal/flats/'
biasfiles=glob.glob(biaspath+'cal*.fit')
darkfiles=glob.glob(darkpath+'cal*.fit')
master_bias_path='Master_Bias.fit'
master_dark_path='Master_Dark_60s.fit'

stand_fname='standard_phot_test1.csv'
clust_fname=obj[1]+'_phot_test1.csv'
model_pname='model_params.csv'

#source and background subfields for centroiding, default is to use 1st alphabetical image name, so choose these accordingly
stand_st=np.array([1642,337])
stand_bg=np.array([1537,447])
clust_st=np.array([448,1613])
clust_bg=np.array([485,1703])

#standard star coords
stc=[1480,210]

#snr limit, fwhm, aperture and annulus params
nsig=25
fwhm=8
phot_ap=12
phot_an_in=15
phot_an_out=20

#standard star official photometric values (apparent mags)
st_b1_simbad=12.772
st_b1_simbad_err=.0009
st_b2_simbad=13.090
st_b2_simbad_err=.0009
st_b3_simbad=13.238
st_b3_simbad_err=.0009

In [4]:
#constructing the master bias
median_bias=sm.mediancombine(biasfiles)
b_header=fits.getheader(biasfiles[0])
mb_name=master_bias_path
fits.writeto(mb_name, median_bias, b_header, overwrite=True)

#the master dark
for i in range(len(darkfiles)):
    sm.bias_subtract(darkfiles[i],master_bias_path,darkpath)
    
bdarks=glob.glob(darkpath+'b_cal*.fit')
median_bdark=sm.mediancombine(bdarks)
bdheader=fits.getheader(bdarks[0])
db_name=master_dark_path
fits.writeto(db_name,median_bdark,bdheader,overwrite=True)

#master flats
bands=filters
for index in range(len(bands)):
    band=bands[index]
    flatpath2=flatpath+band+'band/'
    flatfiles=glob.glob(flatpath2+'cal*.fit')
    for j in range(len(flatfiles)):
        sm.bias_subtract(flatfiles[j],master_bias_path,flatpath2)

for index in range(len(bands)):
    band=bands[index]
    flatpath2=flatpath+band+'band/'
    b_flatfiles=glob.glob(flatpath2+'b_cal*.fit')
    for j in range(len(b_flatfiles)):
        sm.dark_subtract(b_flatfiles[j],master_dark_path,flatpath2)
    
for index in range(len(bands)):
    band=bands[index]
    flatpath2=flatpath+band+'band/'
    db_flatfiles=glob.glob(flatpath2+'db_cal*.fit')
    masterflat=sm.norm_combine_flats(db_flatfiles)
    flatheader=fits.getheader(db_flatfiles[0])
    masterflatname='Master_Flat_'+band+'band.fit'
    fits.writeto(masterflatname,masterflat,flatheader,overwrite=True)

In [4]:
#image cal for each target
for objec in obj:
    #bias subtract
    imgs=glob.glob(impath+objec+'*.fit')
    for current_image in imgs:
        sm.bias_subtract(current_image,master_bias_path,impath)
    
    #get list of b subtracted imgs; dark subtract
    imgs=glob.glob(impath+'b_'+objec+'*.fit')
    for current_image in imgs:
        sm.dark_subtract(current_image,master_dark_path,impath)
        
    #now go thru list of filts; flatfield
    for band in filters:
        dbstand=glob.glob(impath+'db_'+objec+'*'+band+'.fit')
        mflatpath='Master_Flat_'+band+'band.fit'
        for k in range(len(dbstand)):
            sm.flatfield(dbstand[k],mflatpath,impath)

In [6]:
#stacking; by default chooses 1st image by alphabetical order to align to -> B band 1st exposure
refstcs=[stand_st,clust_st]
refbgcs=[stand_bg,clust_bg]
i=0
for objec in obj:
    strefer=refstcs[i]
    bgrefer=refbgcs[i]
    ref_p=glob.glob(impath+'fdb_'+objec+'*.fit')
    ref_p.sort()
    ref_p=ref_p[0]
    sm.stack_image_set(objec,impath,filters,ref_p,strefer,bgrefer)
    i+=1

Using 2022oct27\fdb_standard-0001_B.fit as reference
Stacking:  ['2022oct27\\fdb_standard-0001_B.fit', '2022oct27\\fdb_standard-0002_B.fit', '2022oct27\\fdb_standard-0003_B.fit']


C:\Users\Seth\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1120: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


Stacked B band images
Stacking:  ['2022oct27\\fdb_standard-0001_V.fit', '2022oct27\\fdb_standard-0002_V.fit', '2022oct27\\fdb_standard-0003_V.fit']
Stacked V band images
Stacking:  ['2022oct27\\fdb_standard-0001_R.fit', '2022oct27\\fdb_standard-0002_R.fit', '2022oct27\\fdb_standard-0003_R.fit']
Stacked R band images
Using 2022oct27\fdb_NGC7789-0001_B.fit as reference
Stacking:  ['2022oct27\\fdb_NGC7789-0001_B.fit', '2022oct27\\fdb_NGC7789-0002_B.fit', '2022oct27\\fdb_NGC7789-0003_B.fit', '2022oct27\\fdb_NGC7789-0004_B.fit', '2022oct27\\fdb_NGC7789-0005_B.fit', '2022oct27\\fdb_NGC7789-0006_B.fit', '2022oct27\\fdb_NGC7789-0007_B.fit', '2022oct27\\fdb_NGC7789-0008_B.fit']
Stacked B band images
Stacking:  ['2022oct27\\fdb_NGC7789-0001_V.fit', '2022oct27\\fdb_NGC7789-0002_V.fit', '2022oct27\\fdb_NGC7789-0003_V.fit', '2022oct27\\fdb_NGC7789-0004_V.fit', '2022oct27\\fdb_NGC7789-0005_V.fit', '2022oct27\\fdb_NGC7789-0006_V.fit', '2022oct27\\fdb_NGC7789-0007_V.fit', '2022oct27\\fdb_NGC7789-0008_

In [7]:
#photometry time; first get positions
stand_im=glob.glob(impath+'Stacked_standard/Stacked_image*'+filters[0]+'.fits')[0]
st_x,st_y=sm.starExtractor(stand_im,nsigma_value=nsig,fwhm_value=fwhm)

clust_im=glob.glob(impath+'Stacked_'+obj[1]+'/Stacked_image*'+filters[0]+'.fits')[0]
cl_x,cl_y=sm.starExtractor(clust_im,nsigma_value=nsig,fwhm_value=fwhm)

#define table cols according to filter selection
flux_cols=[filt+'flux' for filt in filters]
efluxcols=[filt+'fluxerr' for filt in filters]

columns = ['id','xcenter', 'ycenter',flux_cols[0],efluxcols[0],flux_cols[1],efluxcols[1],flux_cols[2],efluxcols[2]]

2022oct27/Stacked_standard\Stacked_image_standard_B.reg already exists in this directory. Rename or remove the .reg file and run again.
Number of stars found in  2022oct27/Stacked_standard\Stacked_image_standard_B.fits : 12
2022oct27/Stacked_NGC7789\Stacked_image_NGC7789_B.reg already exists in this directory. Rename or remove the .reg file and run again.
Number of stars found in  2022oct27/Stacked_NGC7789\Stacked_image_NGC7789_B.fits : 586


In [8]:
#standard field flux counting
stand_B=glob.glob(impath+'Stacked_standard/Stacked_image*'+filters[0]+'.fits')[0]
stand_berr=sm.bg_error_estimate(stand_B)
stand_B_tab=sm.measurePhotometry(stand_B, starxy_pos_list=np.c_[st_x, st_y],\
                              aperture_radius=phot_ap, sky_inner=phot_an_in, sky_outer=phot_an_out, error_array=stand_berr)

stand_V=glob.glob(impath+'Stacked_standard/Stacked_image*'+filters[1]+'.fits')[0]
stand_verr=sm.bg_error_estimate(stand_V)
stand_V_tab=sm.measurePhotometry(stand_V, starxy_pos_list=np.c_[st_x, st_y],\
                              aperture_radius=phot_ap, sky_inner=phot_an_in, sky_outer=phot_an_out, error_array=stand_verr)

stand_R=glob.glob(impath+'Stacked_standard/Stacked_image*'+filters[2]+'.fits')[0]
stand_rerr=sm.bg_error_estimate(stand_R)
stand_R_tab=sm.measurePhotometry(stand_R, starxy_pos_list=np.c_[st_x, st_y],\
                              aperture_radius=phot_ap, sky_inner=phot_an_in, sky_outer=phot_an_out, error_array=stand_rerr)

exp_b=fits.getheader(stand_B)['EXPOSURE']
exp_v=fits.getheader(stand_V)['EXPOSURE']
exp_r=fits.getheader(stand_R)['EXPOSURE']

stand_flux=pd.DataFrame(
    {'id'      : stand_B_tab['id'],
     'xcenter' : stand_B_tab['xcenter'],
     'ycenter' : stand_B_tab['ycenter'],
     'Bflux'   : stand_B_tab['bg_subtracted_star_counts']/exp_b,
     'Bfluxerr': stand_B_tab['bg_sub_star_cts_err']/exp_b,
     'Vflux'   : stand_V_tab['bg_subtracted_star_counts']/exp_v,
     'Vfluxerr': stand_V_tab['bg_sub_star_cts_err']/exp_v, 
     'Rflux'   : stand_R_tab['bg_subtracted_star_counts']/exp_r,
     'Rfluxerr': stand_R_tab['bg_sub_star_cts_err']/exp_r}, columns=columns)

#and target cluster flux counts
stand_B=glob.glob(impath+'Stacked_'+obj[1]+'/Stacked_image*'+filters[0]+'.fits')[0]
stand_berr=sm.bg_error_estimate(stand_B)
stand_B_tab=sm.measurePhotometry(stand_B, starxy_pos_list=np.c_[cl_x, cl_y],\
                              aperture_radius=phot_ap, sky_inner=phot_an_in, sky_outer=phot_an_out, error_array=stand_berr)

stand_V=glob.glob(impath+'Stacked_'+obj[1]+'/Stacked_image*'+filters[1]+'.fits')[0]
stand_verr=sm.bg_error_estimate(stand_V)
stand_V_tab=sm.measurePhotometry(stand_V, starxy_pos_list=np.c_[cl_x, cl_y],\
                              aperture_radius=phot_ap, sky_inner=phot_an_in, sky_outer=phot_an_out, error_array=stand_verr)

stand_R=glob.glob(impath+'Stacked_'+obj[1]+'/Stacked_image*'+filters[2]+'.fits')[0]
stand_rerr=sm.bg_error_estimate(stand_R)
stand_R_tab=sm.measurePhotometry(stand_R, starxy_pos_list=np.c_[cl_x, cl_y],\
                              aperture_radius=phot_ap, sky_inner=phot_an_in, sky_outer=phot_an_out, error_array=stand_rerr)

exp_b=fits.getheader(stand_B)['EXPOSURE']
exp_v=fits.getheader(stand_V)['EXPOSURE']
exp_r=fits.getheader(stand_R)['EXPOSURE']

clust_flux=pd.DataFrame(
    {'id'      : stand_B_tab['id'],
     'xcenter' : stand_B_tab['xcenter'],
     'ycenter' : stand_B_tab['ycenter'],
     'Bflux'   : stand_B_tab['bg_subtracted_star_counts']/exp_b,
     'Bfluxerr': stand_B_tab['bg_sub_star_cts_err']/exp_b,
     'Vflux'   : stand_V_tab['bg_subtracted_star_counts']/exp_v,
     'Vfluxerr': stand_V_tab['bg_sub_star_cts_err']/exp_v, 
     'Rflux'   : stand_R_tab['bg_subtracted_star_counts']/exp_r,
     'Rfluxerr': stand_R_tab['bg_sub_star_cts_err']/exp_r}, columns=columns)

stand_flux.head()

Writing the background-only error image:  2022oct27/Stacked_standard\Stacked_image_standard_B_bgerror.fit
Writing the total error image:  2022oct27/Stacked_standard\Stacked_image_standard_B_error.fit
Writing the background-only error image:  2022oct27/Stacked_standard\Stacked_image_standard_V_bgerror.fit
Writing the total error image:  2022oct27/Stacked_standard\Stacked_image_standard_V_error.fit
Writing the background-only error image:  2022oct27/Stacked_standard\Stacked_image_standard_R_bgerror.fit
Writing the total error image:  2022oct27/Stacked_standard\Stacked_image_standard_R_error.fit
Writing the background-only error image:  2022oct27/Stacked_NGC7789\Stacked_image_NGC7789_B_bgerror.fit
Writing the total error image:  2022oct27/Stacked_NGC7789\Stacked_image_NGC7789_B_error.fit
Writing the background-only error image:  2022oct27/Stacked_NGC7789\Stacked_image_NGC7789_V_bgerror.fit
Writing the total error image:  2022oct27/Stacked_NGC7789\Stacked_image_NGC7789_V_error.fit
Writing 

,id,xcenter,ycenter,Bflux,Bfluxerr,Vflux,Vfluxerr,Rflux,Rfluxerr
0,1,262.746805,40.964705,1363.721088,3.443363,2751.102173,5.002543,3735.891580,5.897714
1,2,1487.906392,209.121785,662.112495,2.765264,678.839483,3.504491,561.177296,3.846724
2,3,723.591720,272.229881,5580.516804,6.161122,10371.643183,8.480326,13221.567564,9.796206
3,4,1630.715734,325.692661,419.401865,2.471714,727.751974,3.567517,885.018994,4.129743
4,5,747.672037,392.795417,565.877216,2.634912,1479.385123,4.154743,2349.599808,5.144300


In [9]:
#as long as shifts arent too large and standard field isnt too crowded
#standard star can be located prior to stacking, and located by its coordinates
xst=stand_flux['xcenter']
yst=stand_flux['ycenter']
stand_ind=(xst>stc[0]-25)&(xst<stc[0]+25)&(yst>stc[1]-25)&(yst<stc[1]+25)
print(stand_flux.loc[stand_ind])

   id      xcenter     ycenter       Bflux  Bfluxerr       Vflux  Vfluxerr  \
1   2  1487.906392  209.121785  662.112495  2.765264  678.839483  3.504491   

        Rflux  Rfluxerr  
1  561.177296  3.846724  


In [10]:
#zeropoint calculation using standard star flux counts and official photometry values
standard_star=stand_flux.loc[1]
st_b_inst=-2.5*np.log10(standard_star[flux_cols[0]])
st_b_inst_err=2.5*.434*(standard_star[efluxcols[0]]/standard_star[flux_cols[0]])
b_zp=st_b1_simbad-st_b_inst
b_zp_err=np.sqrt(st_b_inst_err**2 + st_b1_simbad_err**2)

st_v_inst=-2.5*np.log10(standard_star[flux_cols[1]])
st_v_inst_err=2.5*.434*(standard_star[efluxcols[1]]/standard_star[flux_cols[1]])
v_zp=st_b2_simbad-st_v_inst
v_zp_err=np.sqrt(st_v_inst_err**2 + st_b2_simbad_err**2)

st_r_inst=-2.5*np.log10(standard_star[flux_cols[2]])
st_r_inst_err=2.5*.434*(standard_star[efluxcols[2]]/standard_star[flux_cols[2]])
r_zp=st_b3_simbad-st_r_inst
r_zp_err=np.sqrt(st_r_inst_err**2 + st_b3_simbad_err**2)

zeropoints=[b_zp,v_zp,r_zp]
zeropointerrs=[b_zp_err,v_zp_err,r_zp_err]
print(zeropoints)
print(zeropointerrs)

[19.82432945866094, 20.169417735171315, 20.11075023062139]
[0.004619934317004513, 0.005673127880690351, 0.007491649127062433]


In [11]:
magcols=[filt+'_inst' for filt in filters]
emagcols=[filt+'inst_err' for filt in filters]
#instrumental mags
#cluster
clust_flux[magcols[0]]=-2.5*np.log10(np.absolute(clust_flux[flux_cols[0]]))
clust_flux[emagcols[0]]=2.5*.434*(clust_flux[efluxcols[0]]/clust_flux[flux_cols[0]])

clust_flux[magcols[1]]=-2.5*np.log10(np.absolute(clust_flux[flux_cols[1]]))
clust_flux[emagcols[1]]=2.5*.434*(clust_flux[efluxcols[1]]/clust_flux[flux_cols[1]])

clust_flux[magcols[2]]=-2.5*np.log10(np.absolute(clust_flux[flux_cols[2]]))
clust_flux[emagcols[2]]=2.5*.434*(clust_flux[efluxcols[2]]/clust_flux[flux_cols[2]])

#standard
stand_flux[magcols[0]]=-2.5*np.log10(np.absolute(stand_flux[flux_cols[0]]))
stand_flux[emagcols[0]]=2.5*.434*(stand_flux[efluxcols[0]]/stand_flux[flux_cols[0]])

stand_flux[magcols[1]]=-2.5*np.log10(np.absolute(stand_flux[flux_cols[1]]))
stand_flux[emagcols[1]]=2.5*.434*(stand_flux[efluxcols[1]]/stand_flux[flux_cols[1]])

stand_flux[magcols[2]]=-2.5*np.log10(np.absolute(stand_flux[flux_cols[2]]))
stand_flux[emagcols[2]]=2.5*.434*(stand_flux[efluxcols[2]]/stand_flux[flux_cols[2]])

In [12]:
#apparent mags using zeropoints and inst mags
tabs=[stand_flux,clust_flux]
for tab in tabs:
    for i in range(len(zeropoints)):
        filt=filters[i]
        zp_curr=zeropoints[i]
        zp_err_curr=zeropointerrs[i]
        sm.zpcalc(zp_curr,zp_err_curr,filt,tab)

In [13]:
#write phot catalogs
stand_flux.to_csv(stand_fname)
clust_flux.to_csv(clust_fname)